In [1]:
import numpy as np

Let's write a simple model which could capture the following empirical observations
 - Drivers use two perceptual quantities in making a decision: distance (d) and time-to-arrival (TTA)
 - Probability of turning increases with both d and TTA
 - Response time increases with TTA
 - Response time decreases with d

In [2]:
def dx():
    pass